In [1]:
import pandas as pd
import json
import torch
import pickle

/raid/home/kawinm/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
relations_json = json.loads(open('export_relations_19_may_2023.json', encoding='utf-8-sig').read())
nodes_json = json.loads(open('export_nodes_19_may_2023.json', encoding='utf-8-sig').read())

In [3]:
node_map = {}
fin_ind_nodes = {}
node_label_map, label_map, label_id = {}, {}, 1
for nodes in nodes_json:
    node = nodes['n']
    node_map[node['identity']] = node
    if node['labels'][0] == 'Financial_Indicators':
        fin_ind_nodes[node['identity']] = node
    print(node)
    if node['labels'][0] not in label_map:
        label_map[node['labels'][0]] = label_id
        label_id += 1
    node_label_map[node['identity']] = label_map[node['labels'][0]]

{'identity': 0, 'labels': ['Person'], 'properties': {'name': 'Tommy Millner', 'id': 114689399}, 'elementId': '0'}
{'identity': 1, 'labels': ['Person'], 'properties': {'name': 'Claudia Munce', 'id': 114689485}, 'elementId': '1'}
{'identity': 2, 'labels': ['Person'], 'properties': {'name': 'Richelle Parham', 'id': 27452297}, 'elementId': '2'}
{'identity': 3, 'labels': ['Person'], 'properties': {'name': 'Steve Rendle', 'id': 114689617}, 'elementId': '3'}
{'identity': 4, 'labels': ['Person'], 'properties': {'name': 'Eugene Woods', 'id': 114689672}, 'elementId': '4'}
{'identity': 5, 'labels': ['Person'], 'properties': {'name': 'Rudy de Leon', 'id': 7377714}, 'elementId': '5'}
{'identity': 6, 'labels': ['Person'], 'properties': {'name': 'Cecil D. Haney', 'id': 5056036}, 'elementId': '6'}
{'identity': 7, 'labels': ['Person'], 'properties': {'name': 'Mark Malcolm', 'id': 114899342}, 'elementId': '7'}
{'identity': 8, 'labels': ['Person'], 'properties': {'name': 'James Mattis', 'id': 267902}, 'e

In [4]:
node_tensor = torch.ones(6502).long()
for node, label in node_label_map.items():
    node_tensor[node] = label+1
torch.save(node_tensor, 'node_tensor_usa.pt')
print(node_tensor, max(node_tensor))

tensor([2, 2, 2,  ..., 1, 1, 1]) tensor(13)


In [9]:
fin_ind_nodes

{579: {'identity': 579,
  'labels': ['Financial_Indicators'],
  'properties': {'name': 'NON-GAAP EPS', 'id': 'NON-GAAP EPS'},
  'elementId': '579'},
 580: {'identity': 580,
  'labels': ['Financial_Indicators'],
  'properties': {'name': 'REVENUE', 'id': 'REVENUE'},
  'elementId': '580'},
 581: {'identity': 581,
  'labels': ['Financial_Indicators'],
  'properties': {'name': 'EPS', 'id': 'EPS'},
  'elementId': '581'},
 582: {'identity': 582,
  'labels': ['Financial_Indicators'],
  'properties': {'name': 'GUIDANCE', 'id': 'GUIDANCE'},
  'elementId': '582'},
 641: {'identity': 641,
  'labels': ['Financial_Indicators'],
  'properties': {'name': 'LONG-TERM-DEBT-CAPITAL'},
  'elementId': '641'},
 642: {'identity': 642,
  'labels': ['Financial_Indicators'],
  'properties': {'name': 'DEBT-EQUITY-RATIO'},
  'elementId': '642'},
 643: {'identity': 643,
  'labels': ['Financial_Indicators'],
  'properties': {'name': 'GROSS-MARGIN'},
  'elementId': '643'},
 644: {'identity': 644,
  'labels': ['Financ

In [10]:
count = 0
for node_id in range(5025):
    if node_id not in node_map:
        print(node_id, end=" ")
        count += 1
print(end="\n")
print("Missing node id - Count: ", count)

540 541 578 583 584 585 586 587 625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 658 659 661 662 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 697 698 699 700 701 702 703 704 705 706 707 708 709 710 713 714 715 716 717 718 719 720 721 834 835 1099 1100 1386 1387 1397 1398 1399 1400 1401 1402 1459 1460 1558 1559 2798 2799 2972 2973 3212 3213 3757 3758 4093 4094 4191 4192 4403 4404 
Missing node id - Count:  114


In [18]:
fin_ind_relations = {'INCREASE_8Q': 1493,
 'LARGE_INCREASE_QoQ': 72392,
 'LARGE_DECREASE_QoQ': 95533,
 'DECREASE_8Q': 7477,
 'DECREASE_16Q': 3304,
 'INCREASE_YoY': 48726,
 'DECREASE_YoY': 56478,
 'DECREASE_3Y': 12314,
 'INCREASE_3Y': 7224,
 'DECREASE_5Y': 3840,
 'INCREASE_5Y': 1116,
 'INCREASE_16Q': 71}


relation_map, rid = {}, 0
for relations in relations_json:
    relation = relations['r']
    if relation['end'] in fin_ind_nodes and relation['type'] in fin_ind_relations:
        if "INCREASE" in relation['type']:
            relation['type'] = relation['type'].replace("INCREASE", "DECREASE")
        elif "DECREASE" in relation['type']:
            relation['type'] = relation['type'].replace("DECREASE", "INCREASE")
    relation_map[rid] = [relation['start'], relation['end'], relation['type'], relation['properties']]
    rid += 1

In [19]:
print("Number of nodes: {0} and Number of relations: {1}".format(len(node_map), len(relation_map)))

Number of nodes: 4911 and Number of relations: 370567


In [20]:
nasdaq_c_id = {'SBUX': 0, 'VRSK': 1, 'ADBE': 2, 'NVDA': 3, 'AMGN': 4, 'ISRG': 5, 'KLAC': 6, 'AAPL': 7, 'REGN': 8, 'BIDU': 9, 'QCOM': 10, 'XEL': 11, 'MNST': 12, 'CTSH': 13, 'EA': 14, 'WBA': 15, 'ADSK': 16, 'EBAY': 17, 'ILMN': 18, 'NTES': 19, 'SWKS': 20, 'ADP': 21, 'MU': 22, 'AMZN': 23, 'MCHP': 24, 'NXPI': 25, 'FTNT': 26, 'SGEN': 27, 'TXN': 28, 'DXCM': 29, 'AZN': 30, 'VRSN': 31, 'MTCH': 32, 'CHTR': 33, 'AMAT': 34, 'FISV': 35, 'TMUS': 36, 'MSFT': 37, 'CTAS': 38, 'IDXX': 39, 'TSLA': 40, 'MAR': 41, 'AEP': 42, 'NFLX': 43, 'CDNS': 44, 'ALGN': 45, 'MELI': 46, 'GILD': 47, 'SIRI': 48, 'EXC': 49, 'CSCO': 50, 'LULU': 51, 'GOOGL': 52, 'MRVL': 53, 'ASML': 54, 'BIIB': 55, 'GOOG': 56, 'BKNG': 57, 'FAST': 58, 'ORLY': 59, 'ROST': 60, 'VRTX': 61, 'PCAR': 62, 'CMCSA': 63, 'PEP': 64, 'ATVI': 65, 'AMD': 66, 'ANSS': 67, 'AVGO': 68, 'INTU': 69, 'SNPS': 70, 'COST': 71, 'DLTR': 72, 'HON': 73, 'INTC': 74, 'PAYX': 75, 'CSX': 76, 'LRCX': 77, 'ADI': 78, 'MDLZ': 79, 'KDP': 80, 'CPRT': 81, 'ODFL': 82}
sp500_c_id = {'APH': 0, 'PG': 1, 'OMC': 2, 'VRTX': 3, 'EXC': 4, 'KMX': 5, 'STE': 6, 'SPG': 7, 'GE': 8, 'TMO': 9, 'EA': 10, 'CTAS': 11, 'PNR': 12, 'EIX': 13, 'SCHW': 14, 'ADI': 15, 'DLTR': 16, 'EQR': 17, 'GIS': 18, 'TRMB': 19, 'FCX': 20, 'UPS': 21, 'FDX': 22, 'SYY': 23, 'COF': 24, 'LEN': 25, 'WELL': 26, 'HSIC': 27, 'BDX': 28, 'GOOGL': 29, 'BEN': 30, 'LLY': 31, 'VFC': 32, 'WFC': 33, 'WHR': 34, 'UNP': 35, 'FMC': 36, 'GPC': 37, 'ABT': 38, 'IVZ': 39, 'WAT': 40, 'WEC': 41, 'ROK': 42, 'TSLA': 43, 'BRO': 44, 'LNC': 45, 'AME': 46, 'TDY': 47, 'DAL': 48, 'CTRA': 49, 'ROP': 50, 'PENN': 51, 'PEP': 52, 'REGN': 53, 'UNH': 54, 'NFLX': 55, 'DISH': 56, 'DHI': 57, 'ORLY': 58, 'BBWI': 59, 'JPM': 60, 'NWL': 61, 'A': 62, 'PFG': 63, 'MCHP': 64, 'ORCL': 65, 'NEM': 66, 'HBI': 67, 'AMT': 68, 'CDNS': 69, 'RE': 70, 'ED': 71, 'TDG': 72, 'ARE': 73, 'XEL': 74, 'PHM': 75, 'PEG': 76, 'CME': 77, 'CTSH': 78, 'ES': 79, 'AES': 80, 'GRMN': 81, 'PNW': 82, 'FTNT': 83, 'MO': 84, 'ICE': 85, 'MRO': 86, 'PLD': 87, 'DOV': 88, 'BSX': 89, 'ISRG': 90, 'VLO': 91, 'PPL': 92, 'DFS': 93, 'DUK': 94, 'ADP': 95, 'NVDA': 96, 'MAS': 97, 'ZBH': 98, 'C': 99, 'LEG': 100, 'WU': 101, 'IEX': 102, 'NXPI': 103, 'ANSS': 104, 'ADBE': 105, 'LVS': 106, 'ILMN': 107, 'CRL': 108, 'EXPE': 109, 'IFF': 110, 'DVN': 111, 'CNC': 112, 'WRB': 113, 'VTRS': 114, 'CNP': 115, 'MS': 116, 'LNT': 117, 'ROST': 118, 'CAT': 119, 'AMZN': 120, 'GNRC': 121, 'NTAP': 122, 'KEY': 123, 'PWR': 124, 'PRU': 125, 'HES': 126, 'DD': 127, 'F': 128, 'SBUX': 129, 'MLM': 130, 'WST': 131, 'MPWR': 132, 'IRM': 133, 'NVR': 134, 'JNJ': 135, 'AKAM': 136, 'LKQ': 137, 'REG': 138, 'SEE': 139, 'FISV': 140, 'WYNN': 141, 'PVH': 142, 'DPZ': 143, 'QCOM': 144, 'ACN': 145, 'INTU': 146, 'HSY': 147, 'LUV': 148, 'MHK': 149, 'TJX': 150, 'GWW': 151, 'UHS': 152, 'MSI': 153, 'BK': 154, 'ROL': 155, 'AFL': 156, 'EXR': 157, 'SWKS': 158, 'LH': 159, 'NSC': 160, 'VTR': 161, 'GOOG': 162, 'MSFT': 163, 'BBY': 164, 'AEP': 165, 'XRAY': 166, 'GD': 167, 'CVX': 168, 'HST': 169, 'AVGO': 170, 'WMB': 171, 'SIVB': 172, 'HBAN': 173, 'MKC': 174, 'GM': 175, 'TGT': 176, 'AIZ': 177, 'MDLZ': 178, 'IT': 179, 'MCD': 180, 'IPGP': 181, 'EBAY': 182, 'BAX': 183, 'NKE': 184, 'MGM': 185, 'IBM': 186, 'BKNG': 187, 'CAH': 188, 'RF': 189, 'FRT': 190, 'VZ': 191, 'WM': 192, 'TFC': 193, 'AVB': 194, 'MOS': 195, 'TER': 196, 'CCL': 197, 'DXCM': 198, 'AMGN': 199, 'MKTX': 200, 'CMI': 201, 'AJG': 202, 'PSA': 203, 'MA': 204, 'PPG': 205, 'UAL': 206, 'XOM': 207, 'ALB': 208, 'EMR': 209, 'EQIX': 210, 'CI': 211, 'CE': 212, 'AWK': 213, 'TSN': 214, 'ESS': 215, 'FLT': 216, 'SNA': 217, 'AON': 218, 'LDOS': 219, 'MTCH': 220, 'AAL': 221, 'BWA': 222, 'TPR': 223, 'EMN': 224, 'SO': 225, 'ALGN': 226, 'DE': 227, 'PM': 228, 'DXC': 229, 'SHW': 230, 'CMCSA': 231, 'MAR': 232, 'BAC': 233, 'DRI': 234, 'ZION': 235, 'CMS': 236, 'PKG': 237, 'PKI': 238, 'NI': 239, 'ALL': 240, 'ATVI': 241, 'SJM': 242, 'AIG': 243, 'LYV': 244, 'GLW': 245, 'SYK': 246, 'DLR': 247, 'UDR': 248, 'AXP': 249, 'PH': 250, 'TT': 251, 'EVRG': 252, 'WY': 253, 'LRCX': 254, 'ETN': 255, 'BIIB': 256, 'NUE': 257, 'PGR': 258, 'LMT': 259, 'RJF': 260, 'ODFL': 261, 'V': 262, 'RSG': 263, 'COP': 264, 'JBHT': 265, 'HON': 266, 'PTC': 267, 'SLB': 268, 'HPQ': 269, 'GPN': 270, 'FFIV': 271, 'CHTR': 272, 'CMG': 273, 'BKR': 274, 'ZBRA': 275, 'CLX': 276, 'EXPD': 277, 'TSCO': 278, 'SRE': 279, 'AVY': 280, 'LYB': 281, 'PCAR': 282, 'RTX': 283, 'MPC': 284, 'APD': 285, 'EW': 286, 'PFE': 287, 'EOG': 288, 'L': 289, 'COO': 290, 'SNPS': 291, 'USB': 292, 'FITB': 293, 'DHR': 294, 'CINF': 295, 'TTWO': 296, 'MAA': 297, 'GL': 298, 'RMD': 299, 'CMA': 300, 'INTC': 301, 'HUM': 302, 'BA': 303, 'PNC': 304, 'ULTA': 305, 'GPS': 306, 'NTRS': 307, 'BR': 308, 'K': 309, 'TMUS': 310, 'CSX': 311, 'UAA': 312, 'AMD': 313, 'HRL': 314, 'KLAC': 315, 'FAST': 316, 'ALK': 317, 'MET': 318, 'AMAT': 319, 'MCK': 320, 'RL': 321, 'BLK': 322, 'CBRE': 323, 'CL': 324, 'WDC': 325, 'LHX': 326, 'KMI': 327, 'PEAK': 328, 'TFX': 329, 'CVS': 330, 'KR': 331, 'VMC': 332, 'POOL': 333, 'TAP': 334, 'VRSN': 335, 'LOW': 336, 'DIS': 337, 'HD': 338, 'AMP': 339, 'TXT': 340, 'MTD': 341, 'MDT': 342, 'COST': 343, 'FIS': 344, 'FRC': 345, 'MNST': 346, 'CCI': 347, 'KIM': 348, 'D': 349, 'NEE': 350, 'CBOE': 351, 'TROW': 352, 'IP': 353, 'EL': 354, 'SBAC': 355, 'JNPR': 356, 'GS': 357, 'HAS': 358, 'LUMN': 359, 'GILD': 360, 'PXD': 361, 'MMM': 362, 'HII': 363, 'SWK': 364, 'ATO': 365, 'JCI': 366, 'TYL': 367, 'CAG': 368, 'AAPL': 369, 'CF': 370, 'ECL': 371, 'CPRT': 372, 'CRM': 373, 'EFX': 374, 'ADSK': 375, 'YUM': 376, 'DG': 377, 'DTE': 378, 'PAYX': 379, 'DGX': 380, 'STT': 381, 'CB': 382, 'TECH': 383, 'SPGI': 384, 'WMT': 385, 'VNO': 386, 'URI': 387, 'XYL': 388, 'MCO': 389, 'JKHY': 390, 'KO': 391, 'ITW': 392, 'VRSK': 393, 'HIG': 394, 'FE': 395, 'NOC': 396, 'CHRW': 397, 'KMB': 398, 'HCA': 399, 'AEE': 400, 'HAL': 401, 'RHI': 402, 'HOLX': 403, 'MSCI': 404, 'CPB': 405, 'BMY': 406, 'T': 407, 'MU': 408, 'IDXX': 409, 'NRG': 410, 'J': 411, 'LIN': 412, 'WAB': 413, 'CSCO': 414, 'TEL': 415, 'OXY': 416, 'BXP': 417, 'AZO': 418, 'ABC': 419, 'APA': 420, 'OKE': 421, 'AAP': 422, 'CHD': 423, 'O': 424, 'INCY': 425, 'NDAQ': 426, 'TRV': 427, 'STX': 428, 'WBA': 429, 'RCL': 430, 'ADM': 431, 'ETR': 432, 'STZ': 433, 'MRK': 434, 'MMC': 435, 'MTB': 436, 'AOS': 437, 'IPG': 438, 'DVA': 439, 'TXN': 440, 'BIO': 441}

In [21]:
nasdaq_c_id_to_kg_id, sp_c_id_to_kg_id = {}, {}

for k, v in node_map.items():
    if v['labels'][0] == 'Asset':
        if 'ticker' not in v['properties']:
            print(k, v)
            continue
        if v['properties']['ticker'] in nasdaq_c_id:
            nasdaq_c_id_to_kg_id[nasdaq_c_id[v['properties']['ticker']]] = k
        if v['properties']['ticker'] in sp500_c_id:
            sp_c_id_to_kg_id[sp500_c_id[v['properties']['ticker']]] = k

print(len(nasdaq_c_id_to_kg_id), len(nasdaq_c_id))
print(len(sp_c_id_to_kg_id), len(sp500_c_id))

1405 {'identity': 1405, 'labels': ['Asset'], 'properties': {'name': 'BlackRock', 'id': 219635}, 'elementId': '1405'}
1409 {'identity': 1409, 'labels': ['Asset'], 'properties': {'name': 'The Coca-Cola Company', 'id': 3295867}, 'elementId': '1409'}
1420 {'identity': 1420, 'labels': ['Asset'], 'properties': {'name': 'State Street Corporation', 'id': 2037125}, 'elementId': '1420'}
1424 {'identity': 1424, 'labels': ['Asset'], 'properties': {'name': 'T. Rowe Price', 'id': 3511946}, 'elementId': '1424'}
1427 {'identity': 1427, 'labels': ['Asset'], 'properties': {'name': 'Morgan Stanley', 'id': 334204}, 'elementId': '1427'}
1441 {'identity': 1441, 'labels': ['Asset'], 'properties': {'name': 'Northern Trust', 'id': 970695}, 'elementId': '1441'}
1450 {'identity': 1450, 'labels': ['Asset'], 'properties': {'name': 'General Mills', 'id': 629998}, 'elementId': '1450'}
1467 {'identity': 1467, 'labels': ['Asset'], 'properties': {'name': 'JPMorgan Chase', 'id': 192314}, 'elementId': '1467'}
1470 {'iden

In [22]:
# Checking if all the required nodes are present in the KG
for k, v in nasdaq_c_id.items():
    if v not in sp_c_id_to_kg_id:
        print(k)

for k, v in nasdaq_c_id.items():
    if v not in nasdaq_c_id_to_kg_id:
        print(k)

In [23]:
nasdaq_c_id_to_kg_id_list, sp_c_id_to_kg_id_list = [], []

# if nodes are not present, assign them a empty node slot
missing_node_id = 578      # for pnr, cboe, lumn

for i in range(len(sp500_c_id)+ 100):
    if i in nasdaq_c_id_to_kg_id:
        nasdaq_c_id_to_kg_id_list.append(int(nasdaq_c_id_to_kg_id[i]))
    if i in sp_c_id_to_kg_id:
        sp_c_id_to_kg_id_list.append(int(sp_c_id_to_kg_id[i]))

nasdaq_c_id_to_kg_id_list = torch.Tensor(nasdaq_c_id_to_kg_id_list).long()
sp_c_id_to_kg_id_list = torch.Tensor(sp_c_id_to_kg_id_list).long()

print(nasdaq_c_id_to_kg_id_list.shape, sp_c_id_to_kg_id_list.shape)

torch.Size([83]) torch.Size([442])


In [24]:
from pandas import to_datetime
import datetime

temporal_kg = []
relation_id_map, rid = {}, 1
for key, triple in relation_map.items():
    head, tail, relation = triple[0], triple[1], triple[2]

    if relation not in relation_id_map:
        relation_id_map[relation] = rid
        rid += 1
    
    relation = relation_id_map[relation]
    if 'expiry_timestamp' in triple[3]:
        expiry_ts, start_ts = triple[3]['expiry_timestamp'], pd.to_datetime(triple[3]['timestamp'])
        #print(expiry_ts, start_ts, head, tail, relation)
    elif 'timestamp' in triple[3]:
        start_ts = pd.to_datetime(triple[3]['timestamp'])
        expiry_ts = pd.to_datetime(triple[3]['timestamp']) + pd.offsets.Day(1) 
        #print(expiry_ts, start_ts, head, tail, relation)
    else:
        expiry_ts, start_ts = pd.to_datetime('1970-01-01'), pd.to_datetime('1970-01-01')
        #print(expiry_ts, start_ts, head, tail, relation)
        
    temporal_kg.append([head, tail, relation, start_ts, expiry_ts])
    #print(triple)

In [25]:
len(temporal_kg)

370567

In [17]:
total_kg = [[], [], []]

for k, v in relation_map.items():
    total_kg[0].append(int(v[0]))
    total_kg[1].append(int(v[1]))
    total_kg[2].append(int(v[2]))

total_kg[0] = torch.Tensor(total_kg[0])
total_kg[1] = torch.Tensor(total_kg[1])
total_kg[2] = torch.Tensor(total_kg[2])

temporal_total_kg = []
for k, v in temporal_relation_map.items():
    temporal_total_kg.append(v)

In [26]:
df = pd.DataFrame(temporal_kg, columns=['head', 'tail', 'relation', 'timestamp', 'expiry_ts'])
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df['expiry_ts'] = pd.to_datetime(df['expiry_ts'], utc=True)
df = df.sort_values(by=['timestamp'])

In [27]:
start_time = pd.to_datetime('2017-08-05', utc=True)
end_time = pd.to_datetime('2017-08-07', utc=True)
mask = (df['timestamp'] >= start_time) & (df['timestamp'] <= end_time)
plot_df = df.loc[mask]

In [35]:
plot_df

head  relation tail                 timestamp
4577    938        20  581 2017-08-07 00:00:00+00:00
4578    938        20  580 2017-08-07 00:00:00+00:00
5242   1194        20  581 2017-08-07 00:00:00+00:00
23557   762        20  581 2017-08-07 00:00:00+00:00
5243   1194        20  580 2017-08-07 00:00:00+00:00
23558   762        20  580 2017-08-07 00:00:00+00:00
40615  1188        26  600 2017-08-07 00:00:00+00:00
43346  1067        26  600 2017-08-07 00:00:00+00:00

In [36]:
with open("entire_kg.pkl", "wb") as f:
    pickle.dump({'kg': total_kg, 'nasdaq_map':nasdaq_c_id_to_kg_id_list, 'sp_map': sp_c_id_to_kg_id_list}, f)

In [29]:
with open("temporal_kg.pkl", "wb") as f:
    pickle.dump({'temporal_kg': df, 'nasdaq_map':nasdaq_c_id_to_kg_id_list, 'sp_map': sp_c_id_to_kg_id_list, 'relation_id_map': relation_id_map}, f)

In [12]:
import pickle
import torch

with open("temporal_kg.pkl", "rb") as f:
    data = pickle.load(f)['temporal_kg']
    

In [14]:
torch.Tensor([int(x) for x in data['relation'].values]).max()

tensor(37.)